In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("../data/repa_data_20231109.csv")

df.columns

from sklearn.preprocessing import StandardScaler
X = df.drop("is_retracted", axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

import random
random.seed(622)
y = df["is_retracted"]
re_idx = y == "Yes"
re_X = X[re_idx]
re_Y = y[re_idx]
nre_X = X[~re_idx]
nre_Y = y[~re_idx]
re_Y = re_Y.reset_index(drop=True)
nre_Y = nre_Y.reset_index(drop=True)

test_re_idx = random.sample(range(re_X.shape[0]), k=1000)
test_nre_idx = random.sample(range(nre_X.shape[0]), k=1000)

test_re_X = re_X[test_re_idx]
test_re_Y = re_Y[test_re_idx]
train_re_X = np.delete(re_X, test_re_idx, axis=0)
train_re_Y = re_Y.drop(test_re_idx)
test_nre_X = nre_X[test_nre_idx]
test_nre_Y = nre_Y[test_nre_idx]
train_nre_X = np.delete(nre_X, test_nre_idx, axis=0)
train_nre_Y = nre_Y.drop(test_nre_idx)

test_data_X = np.vstack((test_re_X, test_nre_X))
test_data_Y = pd.concat([test_re_Y, test_nre_Y])
train_data_X = np.vstack((train_re_X, train_nre_X))
train_data_Y = pd.concat([train_re_Y, train_nre_Y])

In [6]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(random_state=42).fit(train_data_X)

pval = clf.predict(test_data_X)

In [7]:
test_data_Y.replace({"Yes": 1, "No": -1}, inplace=True)
sum(test_data_Y != pval)

1082

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_data_Y, pval)
print(cm)

[[121 879]
 [203 797]]


In [19]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=2, novelty=True)
chosen_col = [5, 4, 6, 7]
clf.fit(train_data_X[:, chosen_col])
pval = clf.predict(test_data_X[:, chosen_col])
cm = confusion_matrix(test_data_Y, pval)
print(cm)

[[208 792]
 [236 764]]
